In [15]:
import pandas as pd
import numpy as np

from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

In [2]:
df_init = pd.read_csv('data.csv', sep=';')
df_init.head()

,Id,Result,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,...,Feature_91,Feature_92,Feature_93,Feature_94,Feature_95,Feature_96,Feature_97,Feature_98,Feature_99,Feature_100
0,1,2,2,56.0,12,1,7.0,4.0,0,1.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0
1,2,2,2,69.0,19,1,6.0,4.0,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,2,1,66.0,8,1,4.0,4.0,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,2,2,62.0,16,1,NaN,3.0,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,2,2,67.0,30,1,NaN,4.0,0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
df = df_init.copy()

target_name = 'Result'
golden_features = ['Feature_3', 'Feature_4', 'Feature_32', 'Feature_35']
feature_names = df.columns.drop([target_name] + golden_features + ['Id']).tolist()

df.fillna(0, inplace=True)

In [11]:
df.describe()

,Id,Result,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,...,Feature_91,Feature_92,Feature_93,Feature_94,Feature_95,Feature_96,Feature_97,Feature_98,Feature_99,Feature_100
count,143.00000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,...,143.000000,143.000000,143.000000,143.000000,143.000000,143.000000,143.0,143.000000,143.000000,143.0
mean,72.00000,1.685315,1.496503,66.594406,12.958042,1.328671,2.664336,2.839161,0.090909,0.755245,...,0.020979,0.090909,0.153846,0.160839,0.083916,0.300699,0.0,0.041958,0.265734,0.0
std,41.42463,0.466023,0.501745,15.666973,8.433563,0.471381,3.126317,0.801850,0.288490,0.431453,...,0.143818,0.288490,0.362069,0.368674,0.278236,1.156945,0.0,0.201198,0.887718,0.0
min,1.00000,1.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
25%,36.50000,1.000000,1.000000,56.000000,6.000000,1.000000,0.000000,2.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
50%,72.00000,2.000000,1.000000,68.000000,12.000000,1.000000,1.000000,3.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
75%,107.50000,2.000000,2.000000,80.000000,18.000000,2.000000,5.000000,3.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0
max,143.00000,2.000000,2.000000,97.000000,40.000000,2.000000,12.000000,4.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,9.000000,0.0,1.000000,9.000000,0.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 143 entries, 0 to 142
Columns: 102 entries, Id to Feature_100
dtypes: float64(84), int64(18)
memory usage: 114.1 KB


In [12]:

X = df[feature_names]
y = df[target_name]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    shuffle=True,
    test_size=0.1,
    random_state=100,
    stratify=y
)

params_model = {
    'auto_class_weights': 'Balanced',
    'eval_metric': 'Accuracy',
    'early_stopping_rounds': 50,
}

cbr_model = CatBoostClassifier(**params_model)

params_grid = {
    'depth': [4, 6, 8, 10],
    'learning_rate': [0.03, 0.1, 0.3, 0.5],
    'iterations': [100, 200, 400, 600, 800, 1000, 1200],
    'l2_leaf_reg': [2, 2.5, 3, 3.5, 4],
    'bagging_temperature': [1, 1.5, 2, 2.5]
}

rs = cbr_model.randomized_search(
    params_grid,
    X_train,
    y_train,
    n_iter=10,
    shuffle=True,
    stratified=True,
    partition_random_seed=100,
    cv=10,
    calc_cv_statistics=True,
    search_by_train_test_split=True,
    refit=True,
)

print(rs['params'])

Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9728260873
bestIteration = 4

0:	loss: 0.9728261	best: 0.9728261 (0)	total: 415ms	remaining: 3.73s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 1
bestIteration = 17

1:	loss: 1.0000000	best: 1.0000000 (1)	total: 654ms	remaining: 2.62s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8586956525
bestIteration = 11

2:	loss: 0.8586957	best: 1.0000000 (1)	total: 1.11s	remaining: 2.59s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9728260873
bestIteration = 4

3:	loss: 0.9728261	best: 1.0000000 (1)	total: 1.54s	remaining: 2.3s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.9728260873
bestIteration = 15

4:	loss: 0.9728261	best: 1.0000000 (1)	total: 4.38s	remaining: 4.38s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.8315217398
bestIteration = 54

5:	loss: 0.8315217	best: 1.0000000 (1)	total: 9.45s	remaining: 6.3s
Stopped

In [17]:
print(f'Accuracy on train: {round(cbr_model.score(X_train, y_train), 2)}')
print(f'Accuracy on test: {round(cbr_model.score(X_test, y_test), 2)}')

y_test_pred = cbr_model.predict(X_test)
print(f'\nClassification report: \n{classification_report(y_test, y_test_pred)}')
print(f'\nConfusion matrix:\n {confusion_matrix(y_test, y_test_pred)}')

Accuracy on train: 1.0
Accuracy on test: 0.93

Classification report: 
              precision    recall  f1-score   support

           1       0.83      1.00      0.91         5
           2       1.00      0.90      0.95        10

    accuracy                           0.93        15
   macro avg       0.92      0.95      0.93        15
weighted avg       0.94      0.93      0.93        15


Confusion matrix:
 [[5 0]
 [1 9]]


In [14]:

feature_importances = pd.DataFrame(
    zip(X.columns, cbr_model.get_feature_importance()),
    columns=['feature_name', 'importance']
)

feature_importances.sort_values(by='importance', ascending=False, inplace=True)
feature_importances.head(50)

,feature_name,importance
65,Feature_70,10.077668
30,Feature_34,7.433991
26,Feature_29,6.641418
94,Feature_99,5.723146
36,Feature_41,5.409357
27,Feature_30,3.883164
33,Feature_38,3.626536
66,Feature_71,3.544044
41,Feature_46,3.478210
6,Feature_9,3.435480
